In [1]:
!nvidia-smi

Fri Oct 20 11:18:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [34]:
%%writefile roteiro1.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <thrust/random.h>
#include <vector>
#define N 100
using namespace std;

int main() {
  thrust::device_vector<float> x(N);
  thrust::sequence (x.begin(), x.end());

  float media = thrust::reduce(x.begin(), x.end()) / x.size();

  thrust::device_vector<float> mu(N);
  thrust::fill(mu.begin(), mu.end(), media);
  thrust::transform(x.begin(), x.end(), mu.begin(), x.begin(), thrust::minus<float>());
  thrust::transform(x.begin(), x.end(), x.begin(), thrust::square<float>());
  float variance = thrust::reduce(x.begin(), x.end()) / N;
  cout << "variancia: " << variance << std::endl;
  return 0;
}

Overwriting roteiro1.cu


In [35]:
!nvcc -arch=sm_70  -std=c++14 roteiro1.cu -o roteiro1

In [36]:
%%timeit
!./roteiro1


variancia: 833.25
variancia: 833.25
variancia: 833.25
variancia: 833.25
variancia: 833.25
variancia: 833.25
variancia: 833.25
variancia: 833.25
315 ms ± 13.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
%%writefile roteiro1Melhor.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/generate.h>
#include <thrust/sort.h>
#include <thrust/copy.h>
#include <thrust/random.h>
#include <vector>
#define N 100
using namespace std;

int main() {
  thrust::device_vector<float> x(N);
  thrust::sequence (x.begin(), x.end());

  float media = thrust::reduce(x.begin(), x.end()) / x.size();

  //thrust::device_vector<float> mu(N);
  //thrust::fill(mu.begin(), mu.end(), media);
  thrust::constant_iterator<float> mu(media);
  thrust::constant_iterator<float> end= mu + N;
  thrust::transform(x.begin(), x.end(), mu, x.begin(), thrust::minus<float>());
  thrust::transform(x.begin(), x.end(), x.begin(), thrust::square<float>());
  float variance = thrust::reduce(x.begin(), x.end()) / N;
  cout << "variancia: " << variance << std::endl;
  return 0;
}

Overwriting roteiro1Melhor.cu


In [38]:
!nvcc -arch=sm_70  -std=c++14 roteiro1Melhor.cu -o roteiro1Melhor

In [39]:
%%timeit
!./roteiro1Melhor

variancia: 8.25
variancia: 8.25
variancia: 8.25
variancia: 8.25
variancia: 8.25
variancia: 8.25
variancia: 8.25
variancia: 8.25
291 ms ± 33.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
